# Hyperparameter optimization

In [1]:
import pandas as pd
import numpy as np
import torch

torch.cuda.empty_cache()

def read_file(fname: str, correct_labels=True) -> pd.DataFrame:
    """Reads a filename, return df with text and labels."""

    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    if correct_labels:
        offensive_ids = df.labels != "Acceptable speech"
        df.labels[offensive_ids] = 1
        df.labels[~offensive_ids] = 0
    else:
        import warnings
        warnings.warn("Labels not corrected!")
    df = df.drop(columns=["role"])
    return df


def run_hyperparams_optimization(model_name, model_type, lang):
    import torch
    torch.cuda.empty_cache()
    if lang not in {"sl", "hr", "en"}:
        raise AttributeError(f"Language {lang} is not valid")
    eval_file, train_file = f"../data/merged-{lang}.test.tsv" , f"../data/merged-{lang}.train.tsv"
    from simpletransformers.classification import ClassificationModel, ClassificationArgs
    import wandb


    model_args = ClassificationArgs()
    model_args.use_early_stopping = True
    model_args.early_stopping_delta = 0.01
    model_args.early_stopping_metric = "mcc"
    model_args.early_stopping_metric_minimize = False
    model_args.early_stopping_patience = 3
    model_args.evaluate_during_training_steps = 1000

    model_args.evaluate_during_training = True
    model_args.manual_seed = 4
    model_args.use_multiprocessing = True
    model_args.eval_batch_size = 8
    model_args.labels_list = [0, 1]
    model_args.wandb_project = "task3"
    model_args.reprocess_input_data = True
    model_args.overwrite_output_dir = True
    model_args.no_save = True

    sweep_config = {
        "method": "bayes",  # grid, random
        "metric": {"name": "mcc", "goal": "maximize"},
        "parameters": {
            "num_train_epochs":  {"max": 15, "min": 3},
            "learning_rate": {"min": 1e-6, "max": 1e-4},
            "train_batch_size": {"max": 100, "min": 5}
        },
    }

    sweep_id = wandb.sweep(sweep_config, project="task3_"+model_name.replace("/", "_")+model_type)

    train_df = read_file(train_file, correct_labels=True)
    eval_df = read_file(eval_file, correct_labels=True)

    def train():
        # Initialize a new wandb run
        wandb.init()

        # Create a TransformerModel
        model = ClassificationModel(
            model_type,
            model_name,
            use_cuda=True,
            args=model_args,
            sweep_config=wandb.config,
        )
        model.overwrite_output_dir = True
        model.no_save = True
        
        
        # Train the model
        model.train_model(train_df, eval_df=eval_df)

        # Evaluate the model
        model.eval_model(eval_df)

        # Sync wandb
        wandb.join()

    wandb.agent(sweep_id, train, count=50)


In [ ]:
model_name = "xlm-roberta-base"
model_type = "xlmroberta"
language = "en"

run_hyperparams_optimization(model_name, model_type, language)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: ypeq6ds8
Sweep URL: https://wandb.ai/5roop/task3_xlm-roberta-basexlmroberta/sweeps/ypeq6ds8


wandb: Agent Starting Run: mvqg2hej with config:
wandb: 	learning_rate: 7.831626915734913e-05
wandb: 	num_train_epochs: 13
wandb: 	train_batch_size: 79
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 5roop (use `wandb login --relogin` to force relogin)


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(


...

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias

Training loss,0.31558
lr,1e-05
global_step,282
_runtime,115
_timestamp,1631639371
_step,10
tp,665
tn,1147
fp,253
fn,237
mcc,0.55484


Training loss,█▆▅▁▁
lr,▄█▆▃▁
global_step,▁▂▃▄▅▆▇█
_runtime,▁▂▃▄▅▅▆▇███
_timestamp,▁▂▃▄▅▅▆▇███
_step,▁▂▂▃▄▅▅▆▇▇█
tp,█▁▁
tn,▁▇█
fp,█▂▁
fn,▁█▇
mcc,▁▅█


wandb: Agent Starting Run: 8v5ypngp with config:
wandb: 	learning_rate: 6.492744044745257e-05
wandb: 	num_train_epochs: 15
wandb: 	train_batch_size: 91
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias

In [2]:
model_name = "xlm-roberta-large"
model_type = "xlmroberta"
language = "en"

run_hyperparams_optimization(model_name, model_type, language)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: ji6bzb1f
Sweep URL: https://wandb.ai/5roop/task3_xlm-roberta-largexlmroberta/sweeps/ji6bzb1f


wandb: Agent Starting Run: 4cj84xsw with config:
wandb: 	learning_rate: 7.519539080462987e-05
wandb: 	num_train_epochs: 7
wandb: 	train_batch_size: 89
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 5roop (use `wandb login --relogin` to force relogin)


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(


...

Training loss,0.60316
lr,0.0
global_step,2830
_runtime,1066
_timestamp,1631731182
_step,70
tp,676
tn,1171
fp,229
fn,226
mcc,0.58553


Training loss,██▇▇▇█▅▃▆▅▅▅▅▆▅▄▂▄▄▄▃▇▅▃▂▅▃▁▃▃▃▃▄▃▆▃▂▃▃▇
lr,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
tp,▁▆█▇▇▇▇▇█▇▇▇
tn,█▇▁▄▄▅▃▄▂▄▄▄
fp,▁▂█▅▅▄▆▅▇▅▅▅
fn,█▃▁▂▂▂▂▂▁▂▂▂
mcc,▁▇▇█▇███████


wandb: Agent Starting Run: hctu39zp with config:
wandb: 	learning_rate: 5.984024881652892e-06
wandb: 	num_train_epochs: 15
wandb: 	train_batch_size: 92
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
model_name = "distilroberta-base"
model_type = "roberta"
language = "en"

run_hyperparams_optimization(model_name, model_type, language)

Create sweep with ID: o4vv4mee
Sweep URL: https://wandb.ai/5roop/task3_distilroberta-baseroberta/sweeps/o4vv4mee


wandb: Agent Starting Run: u1j44lke with config:
wandb: 	learning_rate: 6.76031027740937e-05
wandb: 	num_train_epochs: 8
wandb: 	train_batch_size: 46
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'cl

Run u1j44lke errored: RuntimeError('CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 31.75 GiB total capacity; 30.28 GiB already allocated; 20.75 MiB free; 30.53 GiB reserved in total by PyTorch)')
wandb: ERROR Run u1j44lke errored: RuntimeError('CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 31.75 GiB total capacity; 30.28 GiB already allocated; 20.75 MiB free; 30.53 GiB reserved in total by PyTorch)')
wandb: Agent Starting Run: p7v8nz3y with config:
wandb: 	learning_rate: 2.716734918698293e-05
wandb: 	num_train_epochs: 12
wandb: 	train_batch_size: 79
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'cl

...

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'cl

Training loss,0.1754
lr,0.0
global_step,1476
_runtime,150
_timestamp,1631713828
_step,35
tp,655
tn,1181
fp,219
fn,247
mcc,0.57309


Training loss,▇▇▄▇▄▇▆█▆▄▃▄▄▄▄▂▇▆▃▄▁▄▅▃▃▂▅▂▂
lr,▅██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
global_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▇███
_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
tp,▃▃█▁
tn,▅▆▁█
fp,▄▃█▁
fn,▆▆▁█
mcc,▁██▅


wandb: Agent Starting Run: n0petes6 with config:
wandb: 	learning_rate: 1.731649200133443e-05
wandb: 	num_train_epochs: 14
wandb: 	train_batch_size: 58
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'cl

In [2]:
model_name = "roberta-base"
model_type = "roberta"
language = "en"

run_hyperparams_optimization(model_name, model_type, language)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: vmews0h8
Sweep URL: https://wandb.ai/5roop/task3_roberta-baseroberta/sweeps/vmews0h8


wandb: Agent Starting Run: 4dq3apwg with config:
wandb: 	learning_rate: 4.069461985137065e-05
wandb: 	num_train_epochs: 11
wandb: 	train_batch_size: 51
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 5roop (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(


...

Training loss,0.16472
lr,0.0
global_step,1068
_runtime,167
_timestamp,1631780608
_step,28
tp,643
tn,1184
fp,216
fn,259
mcc,0.56379


Training loss,█▆▆▇▆▅▆▅▄▇▅▃▄▄▃▂▃▁▂▁▃
lr,▄▇██▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▁▁
global_step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇████
_runtime,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇████
_timestamp,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇████
_step,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
tp,▁▇▆█▇
tn,█▃▄▁▃
fp,▁▆▅█▆
fn,█▂▃▁▂
mcc,▁▇▇██


wandb: Agent Starting Run: 2qc6mo1e with config:
wandb: 	learning_rate: 1.0988705049660534e-05
wandb: 	num_train_epochs: 3
wandb: 	train_batch_size: 30
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

wandb: Ctrl + C detected. Stopping sweep.


#  Fine-tuning

In [9]:
from dataclasses import dataclass, field

@dataclass
class Model:
    """Class for keeping track of model params and results"""
    model_name: str
    model_type: str
    language: str
    accuracies: list = field(default_factory=list)
    f1scores: list = field(default_factory=list)
    num_iters: int = 7
    model_args: dict = field(default_factory=dict)


xlarge = Model(
    "xlm-roberta-large",
    "xlmroberta",
    "en",
    model_args= {
        "num_train_epochs": 9,
        "learning_rate": 0.000141,
        "train_batch_size": 46},
    accuracies = [0.6081668114682884, 0.6081668114682884, 0.6081668114682884, 0.6081668114682884, 0.6081668114682884, 0.6081668114682884, 0.6081668114682884],
    f1scores = [0.37817396002160997, 0.37817396002160997, 0.37817396002160997, 0.37817396002160997, 0.37817396002160997, 0.37817396002160997, 0.37817396002160997]
)


xbase = Model(
    "xlm-roberta-base",
    "xlmroberta",
    "en",
    model_args= {
        "num_train_epochs": 3,
        "learning_rate": 0.000097,
        "train_batch_size": 99},
    accuracies = [0.7697654213727194, 0.760642919200695, 0.7832319721980886, 0.6081668114682884, 0.7780191138140747, 0.7819287576020851, 0.6081668114682884],
    f1scores = [0.7573856527317431, 0.7479285719254002, 0.7690206413905999, 0.37817396002160997, 0.7603538620851371, 0.7698192581497165, 0.37817396002160997]
)


distilroberta = Model(
    "distilroberta-base",
    "roberta",
    "en",
    model_args= {
        "num_train_epochs": 4,
        "learning_rate": 0.0000563,
        "train_batch_size": 65},
    accuracies = [0.8053866203301477, 0.7993049522154648, 0.7966985230234579, 0.7893136403127715, 0.7993049522154648, 0.7971329278887923, 0.7958297132927888],
    f1scores = [0.7938755229581946, 0.788071029574932, 0.7862911960389911, 0.776211651538248, 0.788071029574932, 0.7878714869422752, 0.7833737110821903]
)

roberta = Model(
    "roberta-base",
    "roberta",
    "en",
    model_args= {
        "num_train_epochs": 4,
        "learning_rate": 0.0000354,
        "train_batch_size": 76},
    accuracies = [0.8049522154648132, 0.8023457862728063, 0.7975673327541268, 0.8040834057341443, 0.8001737619461338, 0.8019113814074718, 0.8084274543874891],
    f1scores = [0.7938156134306567, 0.7907003000386662, 0.7856867727912031, 0.7933341946050813, 0.7893417576471337, 0.7884840908486412, 0.7978319119382253],
)


models = [roberta, distilroberta, xbase, xlarge]
models

[Model(model_name='roberta-base', model_type='roberta', language='en', accuracies=[0.8049522154648132, 0.8023457862728063, 0.7975673327541268, 0.8040834057341443, 0.8001737619461338, 0.8019113814074718, 0.8084274543874891], f1scores=[0.7938156134306567, 0.7907003000386662, 0.7856867727912031, 0.7933341946050813, 0.7893417576471337, 0.7884840908486412, 0.7978319119382253], num_iters=7, model_args={'num_train_epochs': 4, 'learning_rate': 3.54e-05, 'train_batch_size': 76}),
 Model(model_name='distilroberta-base', model_type='roberta', language='en', accuracies=[0.8053866203301477, 0.7993049522154648, 0.7966985230234579, 0.7893136403127715, 0.7993049522154648, 0.7971329278887923, 0.7958297132927888], f1scores=[0.7938755229581946, 0.788071029574932, 0.7862911960389911, 0.776211651538248, 0.788071029574932, 0.7878714869422752, 0.7833737110821903], num_iters=7, model_args={'num_train_epochs': 4, 'learning_rate': 5.63e-05, 'train_batch_size': 65}),
 Model(model_name='xlm-roberta-base', model_t

In [10]:
import pandas as pd
import numpy as np
import torch

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


def fine_tune_and_evaluate(
    model_type,
    model_name,
    language,
    model_args,
    num_iters,
                            ):
    import torch
    torch.cuda.empty_cache()
    if language not in {"sl", "hr", "en"}:
        raise AttributeError(f"Language {language} is not valid")
    eval_file, train_file = f"../data/merged-{language}.test.tsv" , f"../data/merged-{language}.train.tsv"
    train = read_file(train_file, correct_labels=True)
    test = read_file(eval_file, correct_labels=True)
    
    from simpletransformers.classification import ClassificationModel
    model_args["overwrite_output_dir"] = True
    model_args["output_dir"] = f"finetuned_models/{model_type}__{model_name.replace('/', '_')}_{language}_current/"
    model_args["silent"] = False
    model_args["save_model_every_epoch"] = False
    model_args["save_steps"] = -1
    #model_args["manual_seed"] = 42
    model = ClassificationModel(
        model_type, model_name, use_cuda=True,
        args=model_args

    )
    
    def copy_best_model(model_type, model_name, language):
        import os
        os.system(f"cp -r finetuned_models/{model_type}__{model_name}_{language}_current finetuned_models/{model_type}__{model_name}_{language}_best")
    accuracies = list()
    f1s = list()
    
    
    model.train_model(train, )
    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["labels"].tolist()
    y_pred = model.predict(list(test["text"].values))[0].tolist()
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro")
    
    accuracies.append(accuracy)
    f1s.append(f1)
    
    for i in range(num_iters-1):
        model = ClassificationModel(model_type, model_name, use_cuda=True, args=model_args)
        model.train_model(train, )
        y_true = test["labels"].tolist()
        y_pred = model.predict(list(test["text"].values))[0].tolist()
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average="macro")
    
        accuracies.append(accuracy)
        f1s.append(f1)
        if accuracy == max(accuracies):
            copy_best_model(model_type, model_name, language)
    print(f"Model: {model_name}, {model_type}, {language=}")
    print(f"Accuracies: {accuracies}")
    print(f"F1 scores: {f1s}")
    return accuracies, f1s


for model in models:
    if model.accuracies != []:
        continue
    accs, f1s = fine_tune_and_evaluate(
    model.model_type,
    model.model_name,
    model.language,
    model.model_args,
    model.num_iters)
    
    model.accuracies = accs
    model.f1scores = f1s
    print(model)

In [11]:
models

[Model(model_name='roberta-base', model_type='roberta', language='en', accuracies=[0.8049522154648132, 0.8023457862728063, 0.7975673327541268, 0.8040834057341443, 0.8001737619461338, 0.8019113814074718, 0.8084274543874891], f1scores=[0.7938156134306567, 0.7907003000386662, 0.7856867727912031, 0.7933341946050813, 0.7893417576471337, 0.7884840908486412, 0.7978319119382253], num_iters=7, model_args={'num_train_epochs': 4, 'learning_rate': 3.54e-05, 'train_batch_size': 76}),
 Model(model_name='distilroberta-base', model_type='roberta', language='en', accuracies=[0.8053866203301477, 0.7993049522154648, 0.7966985230234579, 0.7893136403127715, 0.7993049522154648, 0.7971329278887923, 0.7958297132927888], f1scores=[0.7938755229581946, 0.788071029574932, 0.7862911960389911, 0.776211651538248, 0.788071029574932, 0.7878714869422752, 0.7833737110821903], num_iters=7, model_args={'num_train_epochs': 4, 'learning_rate': 5.63e-05, 'train_batch_size': 65}),
 Model(model_name='xlm-roberta-base', model_t

In [14]:
import numpy as np
print("""
|model name| model type| accuracy | macro f1 score|
| ---      | ---       | ---      | ---           |""",)

for model in models:
    print(f"|{model.model_name}| {model.model_type}| {np.mean(model.accuracies):0.3} +/- {np.std(model.accuracies):0.3}", end="")
    print(f"| {np.mean(model.f1scores):0.3} +/- {np.std(model.f1scores):0.3}|")


|model name| model type| accuracy | macro f1 score|
| ---      | ---       | ---      | ---           |
|roberta-base| roberta| 0.803 +/- 0.00323| 0.791 +/- 0.00372|
|distilroberta-base| roberta| 0.798 +/- 0.00446| 0.786 +/- 0.00502|
|xlm-roberta-base| xlmroberta| 0.727 +/- 0.0756| 0.652 +/- 0.173|
|xlm-roberta-large| xlmroberta| 0.608 +/- 0.0| 0.378 +/- 5.55e-17|


In [23]:
sorted_models = sorted(models, key = lambda model: -1*np.mean(model.accuracies))

best_model = sorted_models.pop(0)
rest_models = sorted_models

from scipy import stats
for model in rest_models:
    # Looking at accs:
    higher = best_model.accuracies
    lower = model.accuracies
    import numpy as np
    wilcoxon_acc = stats.wilcoxon(higher,lower, alternative='greater')[1]
    manwhithney_acc = stats.mannwhitneyu(higher,lower, alternative='greater')[1]
    student_acc = stats.ttest_ind(higher,lower)[1]
    
    # Looking at f1s:
    higher = best_model.f1scores
    lower = model.f1scores
    import numpy as np
    wilcoxon_f1 = stats.wilcoxon(higher,lower, alternative='greater')[1]
    manwhithney_f1 = stats.mannwhitneyu(higher,lower, alternative='greater')[1]
    student_f1 = stats.ttest_ind(higher,lower)[1]

    print(f"""
#### `{best_model.model_name}` vs `{model.model_name}`:

| test | accuracy p-value | macro F1 p-value|
| --- | --- | --- |
|Wilcoxon|{wilcoxon_acc:0.3}|{wilcoxon_f1}|
|Mann Whithney|{manwhithney_acc:0.3}|{manwhithney_f1:0.3}|
|Student t-test | {student_acc:0.3e} | {student_f1:0.3e}|

""")


#### `roberta-base` vs `distilroberta-base`:

| test | accuracy p-value | macro F1 p-value|
| --- | --- | --- |
|Wilcoxon|0.0156|0.0390625|
|Mann Whithney|0.0203|0.0367|
|Student t-test | 3.894e-02 | 7.056e-02|



#### `roberta-base` vs `xlm-roberta-base`:

| test | accuracy p-value | macro F1 p-value|
| --- | --- | --- |
|Wilcoxon|0.00781|0.0078125|
|Mann Whithney|0.00107|0.00107|
|Student t-test | 3.062e-02 | 7.135e-02|



#### `roberta-base` vs `xlm-roberta-large`:

| test | accuracy p-value | macro F1 p-value|
| --- | --- | --- |
|Wilcoxon|0.00781|0.0078125|
|Mann Whithney|0.000529|0.000529|
|Student t-test | 6.386e-21 | 4.070e-24|




# Adding Fasttext to model card comparisons

In [8]:
import pandas as pd
import numpy as np

train_fname = "../data/merged-en.train.tsv"
test_fname = "../data/merged-en.test.tsv"

def prepare_for_fasttext(fname):
    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    offensive_ids = df.labels != "Acceptable speech"

    df.labels[offensive_ids] = "Offensive"
    df.labels[~offensive_ids] = "Acceptable"


    df["fasttextlabel"] = df.labels.apply(lambda s: "__label__"+s+" ")
    df["fasttext_all"] = df.fasttextlabel + df.text
    with open(fname+"fasttext", "w") as f:
        for line in list(df.fasttext_all.values):
            f.write(line+"\n")
    #print(f"Wrote data from {fname} to {fname+'fasttext'}")
    
    return df

for language in "sl hr en".split():
    train_fname = f"../data/merged-{language}.train.tsv"
    test_fname = f"../data/merged-{language}.test.tsv"
    
    prepare_for_fasttext(test_fname)
    prepare_for_fasttext(train_fname)
    
    import fasttext
    model = fasttext.train_supervised(input=train_fname+"fasttext", autotuneValidationFile=test_fname+"fasttext", autotuneDuration=1200)
    
    test = prepare_for_fasttext(test_fname)
    y_pred = np.array(model.predict(list(test.text.values))[0]).reshape(-1).tolist()

    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["fasttextlabel"].apply(lambda s: s.split(" ")[0]).values.tolist()
    
    accuracy = accuracy_score(y_true, y_pred)
    print("Language:", language)
    print(f"Accuracy:  {accuracy:0.3}")
    f1 = f1_score(y_true, y_pred, labels=list(set(y_true)), average="macro")
    print(f"F1 score: {f1:0.3}")
    
    
    print(f"\n|fasttext|{accuracy:0.3}|{f1:0.3}|")

Language: sl
Accuracy:  0.717
F1 score: 0.706


|fasttext|0.717|0.706
Language: hr
Accuracy:  0.775
F1 score: 0.757


|fasttext|0.775|0.757
Language: en
Accuracy:  0.731
F1 score: 0.715


|fasttext|0.731|0.715


In [3]:
import pandas as pd
import numpy as np

train_fname = "../data/merged-en.train.tsv"
test_fname = "../data/merged-en.test.tsv"

def prepare_for_fasttext(fname):
    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    offensive_ids = df.labels != "Acceptable speech"

    df.labels[offensive_ids] = "Offensive"
    df.labels[~offensive_ids] = "Acceptable"


    df["fasttextlabel"] = df.labels.apply(lambda s: "__label__"+s+" ")
    df["fasttext_all"] = df.fasttextlabel + df.text
    with open(fname+"fasttext", "w") as f:
        for line in list(df.fasttext_all.values):
            f.write(line+"\n")
    #print(f"Wrote data from {fname} to {fname+'fasttext'}")
    
    return df

for language in "sl hr en".split():
    train_fname = f"../data/merged-{language}.train.tsv"
    test_fname = f"../data/merged-{language}.test.tsv"
    
    prepare_for_fasttext(test_fname)
    prepare_for_fasttext(train_fname)
    
    import fasttext
    model = fasttext.train_supervised(input=train_fname+"fasttext", autotuneValidationFile=test_fname+"fasttext", autotuneDuration=1200)
    
    test = prepare_for_fasttext(test_fname)
    y_pred = np.array(model.predict(list(test.text.values))[0]).reshape(-1).tolist()

    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["fasttextlabel"].apply(lambda s: s.split(" ")[0]).values.tolist()
    
    accuracy = accuracy_score(y_true, y_pred)
    print("Language:", language)
    print(f"Accuracy:  {accuracy:0.3}")
    f1 = f1_score(y_true, y_pred, labels=list(set(y_true)), average="macro")
    print(f"F1 score: {f1:0.3}")
    
    
    print(f"\n|fasttext|{accuracy:0.3}|{f1:0.3}|")

Language: sl
Accuracy:  0.721
F1 score: 0.711

|fasttext|0.721|0.711|
Language: hr
Accuracy:  0.781
F1 score: 0.766

|fasttext|0.781|0.766|
Language: en
Accuracy:  0.732
F1 score: 0.712

|fasttext|0.732|0.712|


In [8]:
%%bash
for lang in "sl" "en" "hr"
do
file="../data/merged-$lang.train.tsvfasttext"
#echo $file
count=$(wc -l < $file)
for_train=$(bc <<< "0.9 * $count / 1")
for_dev=$(bc  <<< "$count - $for_train")

train_filename="../data/fasttext/merged-$lang.train_1.tsvfasttext"
dev_filename="../data/fasttext/merged-$lang.dev_1.tsvfasttext"

head -n $for_train $file > $train_filename
tail -n $for_dev $file > $dev_filename

echo "Wrote first $for_train lines to $train_filename, wrote last $for_dev lines to $dev_filename."
done


Wrote first 7191 lines to ../data/fasttext/merged-sl.train_1.tsvfasttext, wrote last 799 lines to ../data/fasttext/merged-sl.dev_1.tsvfasttext.
Wrote first 8422 lines to ../data/fasttext/merged-en.train_1.tsvfasttext, wrote last 936 lines to ../data/fasttext/merged-en.dev_1.tsvfasttext.
Wrote first 7965 lines to ../data/fasttext/merged-hr.train_1.tsvfasttext, wrote last 886 lines to ../data/fasttext/merged-hr.dev_1.tsvfasttext.


In [9]:
%%bash
for lang in "sl" "en" "hr"
do
file="../data/merged-$lang.train.tsvfasttext"

count=$(wc -l < $file)
for_train=$(bc <<< "0.9 * $count / 1")
for_dev=$(bc  <<< "$count - $for_train")

train_filename="../data/fasttext/merged-$lang.train_2.tsvfasttext"
dev_filename="../data/fasttext/merged-$lang.dev_2.tsvfasttext"

tail -n $for_train $file > $train_filename
head -n $for_dev $file >  $dev_filename

echo "Wrote last $for_train lines to $train_filename, wrote first $for_dev lines to $dev_filename."
done


Wrote last 7191 lines to ../data/fasttext/merged-sl.train_2.tsvfasttext, wrote first 799 lines to ../data/fasttext/merged-sl.dev_2.tsvfasttext.
Wrote last 8422 lines to ../data/fasttext/merged-en.train_2.tsvfasttext, wrote first 936 lines to ../data/fasttext/merged-en.dev_2.tsvfasttext.
Wrote last 7965 lines to ../data/fasttext/merged-hr.train_2.tsvfasttext, wrote first 886 lines to ../data/fasttext/merged-hr.dev_2.tsvfasttext.


In [3]:
%%bash
for lang in "sl" "en" "hr"
do
file="../data/merged-$lang.train.tsvfasttext"

count=$(wc -l < $file)
for_train=$(bc <<< "0.9 * $count / 1")
for_dev=$(bc  <<< "$count - $for_train")

train_filename="../data/fasttext/merged-$lang.train_3.tsvfasttext"
dev_filename="../data/fasttext/merged-$lang.dev_3.tsvfasttext"
shuffled_filename="../data/fasttext/merged-$lang.shuffled_3.tsvfasttext"
#file_contents=$(sort -R $file)
#tail -n $for_train <(echo $file_contents) > $train_filename
#head -n $for_dev <(echo $file_contents) > $dev_filename

sort -R $file > $shuffled_filename
tail -n $for_train $shuffled_filename > $train_filename
head -n $for_dev $shuffled_filename > $dev_filename
echo "Wrote random $for_train lines to $train_filename, wrote random $for_dev lines to $dev_filename."
rm $shuffled_filename
done


Wrote random 7191 lines to ../data/fasttext/merged-sl.train_3.tsvfasttext, wrote random 799 lines to ../data/fasttext/merged-sl.dev_3.tsvfasttext.
Wrote random 8422 lines to ../data/fasttext/merged-en.train_3.tsvfasttext, wrote random 936 lines to ../data/fasttext/merged-en.dev_3.tsvfasttext.
Wrote random 7965 lines to ../data/fasttext/merged-hr.train_3.tsvfasttext, wrote random 886 lines to ../data/fasttext/merged-hr.dev_3.tsvfasttext.


In [1]:
import pandas as pd
import numpy as np


def prepare_for_fasttext(fname):
    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    offensive_ids = df.labels != "Acceptable speech"

    df.labels[offensive_ids] = "Offensive"
    df.labels[~offensive_ids] = "Acceptable"


    df["fasttextlabel"] = df.labels.apply(lambda s: "__label__"+s+" ")
    df["fasttext_all"] = df.fasttextlabel + df.text
    with open(fname+"fasttext", "w") as f:
        for line in list(df.fasttext_all.values):
            f.write(line+"\n")
    #print(f"Wrote data from {fname} to {fname+'fasttext'}")
    
    return df
def read_test_file(fname):
    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    offensive_ids = ~df.text.str.contains("Acceptable")
    df["labels"] = ""
    df.loc[offensive_ids, "labels"] = "Offensive"
    df.loc[~offensive_ids, "labels"] = "Acceptable"


    df["fasttextlabel"] = df.labels.apply(lambda s: "__label__"+s+" ")
    df["fasttext_all"] = df.fasttextlabel + df.text

    return df

#for strategy in (1,2,3):
for strategy in (3,):
    for language in "sl hr en".split():
        train_fname = f"../data/fasttext/merged-{language}.train_{strategy}.tsv"+"fasttext"
        dev_fname = f"../data/fasttext/merged-{language}.dev_{strategy}.tsv"+"fasttext"
        test_fname = f"../data/fasttext/merged-{language}.test.tsv"+"fasttext"
        test = read_test_file(test_fname)

        import fasttext
        model = fasttext.train_supervised(input=train_fname, autotuneValidationFile=dev_fname, autotuneDuration=600)
        
        
        y_pred = np.array(model.predict(list(test.text.values))[0]).reshape(-1).tolist()

        from sklearn.metrics import accuracy_score, f1_score
        y_true = test.fasttextlabel.apply(lambda s: s.split(" ")[0]).values.tolist()
        
        accuracy = accuracy_score(y_true, y_pred)
        #print(f"Strategy number: {strategy}")
        #print("Language:", language)
        #print(f"Accuracy:  {accuracy:0.3}")
        f1 = f1_score(y_true, y_pred, labels=list(set(y_true)), average="macro")
        #print(f"F1 score: {f1:0.3}")
        print(f"|{strategy=}|{language}|{accuracy:0.3}|{f1:0.3}|")

|strategy=3|sl|0.708|0.696|
|strategy=3|hr|0.765|0.749|
|strategy=3|en|0.718|0.702|


In [1]:
import numpy as np

In [12]:
datadict = {
    "hr": {
        "acc": [0.773, 0.772, 0.777, 0.770, 0.761, 0.771],
        "f1s": np.array([759, 757, 761, 750, 744, 754]) / 1000,
    },
    "sl": {
        "acc": np.array([705, 706, 709, 713, 710, 711]) / 1000,
        "f1s": np.array([695, 699, 701, 706, 700, 704]) / 1000,
    },
    "en": {
        "acc": np.array([725, 730, 721, 728, 723, 722]) / 1000,
        "f1s": np.array([709, 713, 702, 709, 703, 703]) / 1000,
    },
}

In [22]:
for lang, data in datadict.items():

    acc = data["acc"]
    acc_mean = np.mean(acc)
    acc_std = np.std(acc)
    
    f1s = data["f1s"]
    f_mean = np.mean(f1s)
    f_std = np.std(f1s)
    print(f"|{lang}|{acc_mean:0.3} +/- {acc_std:0.3}|{f_mean:0.3} +/- {f_std:0.3}|")


|hr|0.771 +/- 0.00485|0.754 +/- 0.00576|
|sl|0.709 +/- 0.00277|0.701 +/- 0.00353|
|en|0.725 +/- 0.00324|0.707 +/- 0.00407|


In [4]:
%load_ext blackcellmagic